In [1]:
import pandas as pd
import numpy as np
import pickle
import argparse
import os
import random

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch

from utils import *

In [2]:
def reproducibility(seed: int):
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.cuda.manual_seed(seed)
reproducibility(3)

In [3]:
# set device to cpu or cuda
device = torch.device('cpu')

if(torch.cuda.is_available()):
    device = torch.device('cuda:0')
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")

Device set to : NVIDIA TITAN RTX


In [4]:

def args_create():
    # @title Arguments
    parser = argparse.ArgumentParser(description='Actor Critic')

    parser.add_argument('--data', default="/data1/sobhan/CEU", type=str, help='Dataset Path')
    parser.add_argument('--episodes', default=29, type=int, metavar='N', help='Number of episodes for training agent.')
    parser.add_argument('--seed', default=3, type=int, help='Seed for reproducibility')
    parser.add_argument('--evaluate', default=True, type=bool, help='Evaluation or Not')


    # Env Properties
    parser.add_argument('--a_control_size', default=50, type=int, help='Attack Control group size')
    parser.add_argument('--b_control_size', default=50, type=int, help='Beacon Control group size')
    parser.add_argument('--gene_size', default=100000, type=int, help='States gene size')
    parser.add_argument('--beacon_size', default=30, type=int, help='Beacon population size')
    parser.add_argument('--victim_prob', default=1, type=float, help='Victim inside beacon or not!')
    parser.add_argument('--max_queries', default=1000, type=int, help='Maximum queries per episode')
    parser.add_argument('--binary', default=True, type=bool, help='Binary queries')

    parser.add_argument('--user_risk', default=0.2, type=float, help='Risk Level for End User')



    parser.add_argument('--attacker_type', default="optimal", choices=["random", "optimal", "agent"], type=str, help='Type of the attacker')
    parser.add_argument('--beacon_type', default="truth", choices=["urandom", "random", "agent", "truth", "beacon_strategy"], type=str, help='Type of the beacon')

    parser.add_argument('--beacon_agent', default="td", choices=["td", "ppo"], type=str, help='Type of the beacon')

    parser.add_argument('--pop_reset_freq', default=100000000, type=int, help='Reset Population Frequency (Epochs)')
    parser.add_argument('--plot-freq', default=1, type=int, metavar='N', help='Plot Frequencies')
    parser.add_argument('--print-freq', default=100, type=int, metavar='N', help='Plot Frequencies')


    # utils
    parser.add_argument('--resume', default=None, type=str, metavar='PATH', help='path to latest checkpoint (default: none)')
    parser.add_argument('--results-dir', default='./results/simulation', type=str, metavar='PATH', help='path to cache (default: none)')

    # args = parser.parse_args()  # running in command line
    args = parser.parse_args('')  # running in ipynb

    args.results_dir = os.path.join(args.results_dir, "run"+str(len(os.listdir(args.results_dir))))
    os.makedirs(args.results_dir)
    os.makedirs(args.results_dir+"/logs")
    os.makedirs(args.results_dir+"/rewards")
    os.makedirs(args.results_dir+"/indrewards")
    os.makedirs(args.results_dir+"/actions")
    os.makedirs(args.results_dir+"/pvalues")

    args.device = device

    print(args)
    return args


In [5]:
# CEU Beacon - it contains 164 people in total which we will divide into groups to experiment
beacon = pd.read_csv(os.path.join("/data1/sobhan/CEU", "Beacon_164.txt"), index_col=0, delim_whitespace=True)
# Reference genome, i.e. the genome that has no SNPs, all major allele pairs for each position
reference = pickle.load(open(os.path.join("/data1/sobhan/CEU", "reference.pickle"),"rb"))
# Binary representation of the beacon; 0: no SNP (i.e. no mutation) 1: SNP (i.e. mutation)
binary = np.logical_and(beacon.values != reference, beacon.values != "NN").astype(int)

In [5]:
import os
import joblib

# Cache file path
cache_path = "/data6/sobhan/Beacons/dataset/binary_cache.joblib"

# Check if the cached file exists
if os.path.exists(cache_path):
    print("Exists")
    # Load from cache
    binary = joblib.load(cache_path)
else:
    # If cache doesn't exist, process and save to cache
    beacon = pd.read_csv(os.path.join("/mnt/kerem/CEU", "Beacon_164.txt"), index_col=0, delim_whitespace=True)
    reference = pickle.load(open(os.path.join("/mnt/kerem/CEU", "reference.pickle"), "rb"))
    binary = np.logical_and(beacon.values != reference, beacon.values != "NN").astype(int)
    
    # Save the processed binary data to cache for future use
    joblib.dump(binary, cache_path)

Exists


In [6]:
# Table that contains MAF (minor allele frequency) values for each position. 
maf = pd.read_csv(os.path.join("/data1/sobhan/CEU", "MAF.txt"), index_col=0, delim_whitespace=True)
maf.rename(columns = {'referenceAllele':'major', 'referenceAlleleFrequency':'major_freq', 
                      'otherAllele':'minor', 'otherAlleleFrequency':'minor_freq'}, inplace = True)
maf["maf"] = np.round(maf["maf"].values, 3)
# Same variable with sorted maf values
sorted_maf = maf.sort_values(by='maf')
# Extracting column to an array for future use
maf_values = maf["maf"].values

binary = binary.T
binary.shape #(164, 4029840)

(164, 4029840)

In [7]:
from ppo import PPO
from td import TD3
%load_ext autoreload
%autoreload 2

from env import Env

args=args_create()


def simulate(args, beacon_type, attacker_type, attacker_resume=None, beacon_resume=None):
    args.beacon_type = beacon_type 
    args.attacker_type = attacker_type 
    # args.beacon_agent = beacon_agent 
    env = Env(args, maf_values, binary)
    ################ PPO hyperparameters ################
    K_epochs = 300         # update policy for K epochs
    eps_clip = 0.1           # clip parameter for PPO
    gamma = 0.99                # discount factor

    lr_actor = 0.0001      # learning rate for actor network
    lr_critic = 0.0001        # learning rate for critic network

    i_episode = 0

    attacker_state_dim = 10
    attacker_action_dim = 12

    beacon_rewards = []
    attacker_rewards = []
    privacies = []
    utilities = []
    utilities2 = []


    if attacker_type == "agent":
        attacker_agent = PPO(attacker_state_dim, attacker_action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, False, None)
        attacker_agent.load(attacker_resume)

    if beacon_type == "agent":
        state_dim = 18
        action_dim = 1
        beacon_agent = TD3(state_dim, action_dim, max_action=1)
        beacon_agent.load(beacon_resume)

    while i_episode < args.episodes:
        beacon_rewardss = []
        attacker_rewardss = []
        privaciess = []
        utilitiess = []
        utilitiess2 = []

        for t in range(1, args.max_queries+1):
            if attacker_type == "agent" and beacon_type == "agent":
                beacon_state, rewards, done, pu  = env.step(attacker_agent=attacker_agent, beacon_agent=beacon_agent)
            elif attacker_type == "agent":
                beacon_state, rewards, done, pu  = env.step(attacker_agent=attacker_agent)
            elif beacon_type == "agent":
                beacon_state, rewards, done, pu  = env.step(beacon_agent=beacon_agent)
            else:
                beacon_state, rewards, done, pu  = env.step()

            # print(beacon_state[0])

            beacon_rewardss.append(rewards[0])
            attacker_rewardss.append(rewards[1])
            privaciess.append(pu[0])
            utilitiess.append(pu[1])
            utilitiess2.append(beacon_state[0][-1])

            if done:
                break
        
        beacon_rewards.append(beacon_rewardss)
        attacker_rewards.append(attacker_rewardss)
        privacies.append(privaciess)
        utilities.append(utilitiess)
        utilities2.append(utilitiess2)


        print("Victim: {} \t Current Episode Reward : {}".format(env.victim_id, rewards[1]))
        env.reset()
        i_episode += 1
    return beacon_rewards, attacker_rewards, privacies, utilities, utilities2

Device set to : NVIDIA TITAN RTX
Device set to : NVIDIA TITAN RTX
Namespace(data='/data1/sobhan/CEU', episodes=29, seed=3, evaluate=True, a_control_size=50, b_control_size=50, gene_size=100000, beacon_size=30, victim_prob=1, max_queries=1000, binary=True, user_risk=0.2, attacker_type='optimal', beacon_type='truth', beacon_agent='td', pop_reset_freq=100000000, plot_freq=1, print_freq=100, resume=None, results_dir='./results/simulation/run188', device=device(type='cuda', index=0))


In [9]:
evaluations = [
        {
        "beacon_type": "agent",
        "attacker_type": "agent",
        "beacon_resume": "/data6/sobhan/Beacons/results/train/run77/weights",
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },
    {
        "beacon_type": "qbudget",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },
    {
        "beacon_type": "truth",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },
    {
        "beacon_type": "baseline",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },

    {
        "beacon_type": "strategic",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },
    {
        "beacon_type": "random",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },
]

In [ ]:
evaluations = [
    {
        "beacon_type": "qbudget",
        "attacker_type": "optimal",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },
    {
        "beacon_type": "truth",
        "attacker_type": "optimal",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },
    {
        "beacon_type": "baseline",
        "attacker_type": "optimal",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },
    {
        "beacon_type": "strategic",
        "attacker_type": "optimal",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },
    {
        "beacon_type": "random",
        "attacker_type": "optimal",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },
    {
        "beacon_type": "qbudget",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },
    {
        "beacon_type": "truth",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },
    {
        "beacon_type": "baseline",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },
    {
        "beacon_type": "strategic",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },
    {
        "beacon_type": "random",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run86/weights/PPO_0.pth"
    },
]

In [10]:
evaluations = [
    {
        "beacon_type": "agent",
        "attacker_type": "agent",
        "beacon_resume": "/data6/sobhan/Beacons/results/train/run77/weights",
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run99/weights/25000/PPO_0.pth"
    },
    {
        "beacon_type": "agent",
        "attacker_type": "agent",
        "beacon_resume": "/data6/sobhan/Beacons/results/train/run99/weights/25000",
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run99/weights/25000/PPO_0.pth"
    },
    {
        "beacon_type": "qbudget",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run99/weights/25000/PPO_0.pth"
    },
    {
        "beacon_type": "truth",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run99/weights/25000/PPO_0.pth"
    },
    {
        "beacon_type": "baseline",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run99/weights/25000/PPO_0.pth"
    },

    {
        "beacon_type": "strategic",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run99/weights/25000/PPO_0.pth"
    },
    {
        "beacon_type": "random",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run99/weights/25000/PPO_0.pth"
    },
]

In [8]:

evaluations = [
    {
        "beacon_type": "agent",
        "attacker_type": "agent",
        "beacon_resume": "/data6/sobhan/Beacons/results/train/run109/weights",
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run106/weights/15000/PPO_0.pth",
        "user_risk": 1
    },
    {
        "beacon_type": "agent",
        "attacker_type": "agent",
        "beacon_resume": "/data6/sobhan/Beacons/results/train/run106/weights/15000",
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run106/weights/15000/PPO_0.pth",
        "user_risk": 1
    },
    {
        "beacon_type": "qbudget",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run106/weights/15000/PPO_0.pth",
        "user_risk": 1
    },
    {
        "beacon_type": "truth",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run106/weights/15000/PPO_0.pth",
        "user_risk": 1
    },
    {
        "beacon_type": "baseline",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run106/weights/15000/PPO_0.pth",
        "user_risk": 1
    },

    {
        "beacon_type": "strategic",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run106/weights/15000/PPO_0.pth",
        "user_risk": 1
    },
    {
        "beacon_type": "random",
        "attacker_type": "agent",
        "beacon_resume": None,
        "attacker_resume": "/data6/sobhan/Beacons/results/train/run106/weights/15000/PPO_0.pth",
        "user_risk": 1
    },
]


In [16]:
beacon_rewards=[]
attacker_rewards=[]
privacies=[]
utilities=[]
utilities2=[]


for eval in evaluations:
    print(eval)
    res = simulate(args, beacon_type=eval["beacon_type"], attacker_type=eval["attacker_type"], beacon_resume=eval['beacon_resume'], attacker_resume=eval["attacker_resume"])
    beacon_rewards.append(res[0])
    attacker_rewards.append(res[1])
    privacies.append(res[2])
    utilities.append(res[3])
    utilities2.append(res[4])
    

{'beacon_type': 'agent', 'attacker_type': 'agent', 'beacon_resume': '/data6/sobhan/Beacons/results/train/run109/weights', 'attacker_resume': '/data6/sobhan/Beacons/results/train/run106/weights/15000/PPO_0.pth', 'user_risk': 1}
./results/simulation/run188/logs/beacon_log.csv
./results/simulation/run188/logs/beacon_control_log.csv
./results/simulation/run188/logs/attacker_log.csv
./results/simulation/run188/logs/attacker_control_log.csv
==================================⬇️⬇️⬇️⬇️Episode: 1⬇️⬇️⬇️⬇️==============================
ATTACKER TYPE:  agent
BEACON TYPE:  agent
-------------------------
--------------------------------Query: 1---------------------------------
Attacker Action: Position 67837 with MAF: 0.2 and SNP: 0.0 and LRT: -0.0
Beacon Action: 1.0
Beacon State: [tensor(0.2000), tensor(0.), tensor(0.), tensor(-1.9073e-06), tensor(-4.4505e-07), tensor(0.), tensor(1.5077), tensor(1.), tensor(0.8600), tensor(0.1400), tensor(0.), tensor(0.), tensor(0.), tensor(-1.9073e-06), tensor(-2.

In [17]:
import pickle
data_dict = {
    'beacon_rewards': beacon_rewards,
    'attacker_rewards': attacker_rewards,
    'privacies': privacies,
    'utilities': utilities,
    'utilities2': utilities2
}

# Save the dictionary to a pickle file
with open('/data6/sobhan/Beacons/results/binary/second_GBA.pkl', 'wb') as f:
    pickle.dump(data_dict, f)

In [ ]:
import numpy as np
%load_ext autoreload
%autoreload 2
from utils import plot_violinplot, plot_boxplot, line_plot, scatter_plot, line_and_bar_plot, plot_panels, line_and_bar_plot2

def pad_to_max_length(data, max_len=1000, value=np.nan):
    padded_data = [[sublst + [value] * (max_len - len(sublst)) for sublst in lst] for lst in data]
    return np.array(padded_data)

queries = [0, 99, 199, 299, 399, 499, 599, 699, 799, 899, 999]
methods = [0, 1, 3, 4, 2, 6, 5]

In [ ]:
labels = ["OBD", "GBD", "Honest Beacon", "Baseline (p=0.01)", "Query Budget (p=0.05)", "Random ($\epsilon$=0.3)",   "Strategic ($\epsilon$=0.1)"] 
# labels = [ "Query Budget (p=0.05)", "Truthful", "Baseline (p=0.01)", "OBD", "SF (e=0.1)", "Random (e=0.3)"] 

# labels = ["Truth", "Agent-Optimal Attacker", "Agent-Agent Attacker", "Random"]  # Define your labels
# labels = ["Optimal Attacker-Beacon Agent A", "Agent Attacker-Beacon Agent A", "Optimal Attacker-Beacon Agent B", "Agent Attacker-Beacon Agent B",]  # Define your labels

In [2]:
line_data_list = []  # your line data for each plot
bar_data_list = []   

In [ ]:
import pickle

# Load the data from the pickle file
with open('/data6/sobhan/Beacons/results/optimal.pkl', 'rb') as f:
    data_dict = pickle.load(f)

utilities2 = data_dict['utilities2']
beacon_rewards = data_dict['beacon_rewards']
padded_utilities = pad_to_max_length(utilities2)[methods]
padded_beacon_rewards = pad_to_max_length(beacon_rewards)[methods]
nan_count = np.isnan(padded_beacon_rewards).sum(axis=1)
line_data_list.append(np.nanmean(padded_utilities, axis=1)[:, queries])
bar_data_list.append(nan_count[:,queries])

# Load the data from the pickle file
with open('/data6/sobhan/Beacons/results/SBA.pkl', 'rb') as f:
    data_dict = pickle.load(f)

utilities2 = data_dict['utilities2']
beacon_rewards = data_dict['beacon_rewards']
padded_utilities = pad_to_max_length(utilities2)[methods]
padded_beacon_rewards = pad_to_max_length(beacon_rewards)[methods]
nan_count = np.isnan(padded_beacon_rewards).sum(axis=1)
line_data_list.append(np.nanmean(padded_utilities, axis=1)[:, queries])
bar_data_list.append(nan_count[:,queries])

# Load the data from the pickle file
with open('/data6/sobhan/Beacons/results/GBA.pkl', 'rb') as f:
    data_dict = pickle.load(f)

utilities2 = data_dict['utilities2']
beacon_rewards = data_dict['beacon_rewards']
padded_utilities = pad_to_max_length(utilities2)[methods]
padded_beacon_rewards = pad_to_max_length(beacon_rewards)[methods]
nan_count = np.isnan(padded_beacon_rewards).sum(axis=1)
line_data_list.append(np.nanmean(padded_utilities, axis=1)[:, queries])
bar_data_list.append(nan_count[:,queries])

line_data_list.append(np.nanmean(padded_utilities, axis=1)[:, queries])
bar_data_list.append(nan_count[:,queries])


In [ ]:
line_data_list.append(np.nanmean(padded_utilities, axis=1)[:, queries])
bar_data_list.append(nan_count[:,queries])

In [ ]:
len(line_data_list)

In [ ]:

line_and_bar_plot2(line_data_list[0], bar_data_list[0], labels, title="Mean of Beacon Utility Level Among 30 Attacks", ylabel_line="Utility Level", ylabel_bar="Num of Predicted")

In [ ]:

line_and_bar_plot(line_data_list[0], bar_data_list[0], labels, title="Mean of Beacon Utility Level Among 30 Attacks", ylabel_line="Utility Level", ylabel_bar="Num of Predicted")

In [ ]:
plot_panels(line_data_list, bar_data_list, labels, ylabel_line="Utility Level", ylabel_bar="Num of Predicted")

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

def plot_four_panels(line_data_list, bar_data_list, labels_list, titles, ylabel_line, ylabel_bar):
    # Generate each plot and save as images
    image_filenames = []
    for i in range(4):
        plt.figure()  # Create a new figure for each plot
        line_and_bar_plot(line_data_list[i], bar_data_list[i], labels_list[i], titles[i], ylabel_line, ylabel_bar)
        filename = f"plot_{i}.png"
        plt.savefig(filename)  # Save each plot as an image
        image_filenames.append(filename)
        plt.close()  # Close the figure to avoid displaying

    # Now load these images and plot them in a 2x2 grid
    fig, axs = plt.subplots(2, 2, figsize=(20, 20))

    for i, ax in enumerate(axs.flat):
        img = Image.open(image_filenames[i])
        ax.imshow(img)
        ax.axis('off')  # Hide the axes

        # Add subplot label (A, B, C, D)
        ax.text(-0.1, 1.1, chr(65+i), transform=ax.transAxes, size=20, weight='bold')

    plt.tight_layout()
    plt.show()

import numpy as np

# Generate random data for the line plot (e.g., 4 classes, 11 query points)
line_data_1 = np.random.rand(7, 11)
line_data_2 = np.random.rand(7, 11)
line_data_3 = np.random.rand(7, 11)
line_data_4 = np.random.rand(7, 11)

# Generate random data for the bar plot (e.g., 4 classes, 11 query points)
bar_data_1 = np.random.randint(1, 12, size=(7, 11))
bar_data_2 = np.random.randint(1, 12, size=(7, 11))
bar_data_3 = np.random.randint(1, 12, size=(7, 11))
bar_data_4 = np.random.randint(1, 12, size=(7, 11))

# Define labels for the 7 classes
labels_1 = labels_2 = labels_3 = labels_4 = ['OBD', 'GBD', 'Truthful', 'Baseline (p=0.01)', 'Query Budget (p=0.05)', 'Random (e=0.3)', 'Strategic (e=0.1)']

# Define titles for each plot
titles = ["Plot 1", "Plot 2", "Plot 3", "Plot 4"]

# Y-axis labels for line and bar plots
ylabel_line = "Utility Level"
ylabel_bar = "Number of Predicted"

# Use the provided function to create the 2x2 grid of plots
plot_four_panels(
    [line_data_1, line_data_2, line_data_3, line_data_4],
    [bar_data_1, bar_data_2, bar_data_3, bar_data_4],
    [labels_1, labels_2, labels_3, labels_4],
    titles,
    ylabel_line,
    ylabel_bar
)

In [ ]:
import pickle

# Load the data from the pickle file
with open('/data6/sobhan/Beacons/results/optimal.pkl', 'rb') as f:
    data_dict = pickle.load(f)

# Access each array
beacon_rewards = data_dict['beacon_rewards']
attacker_rewards = data_dict['attacker_rewards']
privacies = data_dict['privacies']
utilities = data_dict['utilities']
utilities2 = data_dict['utilities2']

methods = [0, 1, 3, 4, 2, 6, 5]

padded_beacon_rewards = pad_to_max_length(beacon_rewards)[methods]
padded_attacker_rewards = pad_to_max_length(attacker_rewards)[methods]
padded_privacies = pad_to_max_length(privacies)[methods]
padded_utilities = pad_to_max_length(utilities2)[methods]
# labels = [ "SBA - OBD", "GBA - OBD", "SBA - GBD", "GBA - GBD",] 
nan_count = np.isnan(padded_beacon_rewards).sum(axis=1)

line_and_bar_plot(np.nanmean(padded_utilities, axis=1)[:, queries], nan_count[:,queries], labels, title="Mean of Beacon Utility Level Among 30 Attacks Against Optimal Attacker", ylabel_line="Utility Level", ylabel_bar="Num of Predicted")

In [ ]:
import pickle

# Load the data from the pickle file
with open('/data6/sobhan/Beacons/results/optimal01.pkl', 'rb') as f:
    data_dict = pickle.load(f)

# Access each array
beacon_rewards = data_dict['beacon_rewards']
attacker_rewards = data_dict['attacker_rewards']
privacies = data_dict['privacies']
utilities = data_dict['utilities']
utilities2 = data_dict['utilities2']

padded_beacon_rewards = pad_to_max_length(beacon_rewards)[methods]
padded_attacker_rewards = pad_to_max_length(attacker_rewards)[methods]
padded_privacies = pad_to_max_length(privacies)[methods]
padded_utilities = pad_to_max_length(utilities2)[methods]
# labels = [ "SBA - OBD", "GBA - OBD", "SBA - GBD", "GBA - GBD",] 
nan_count = np.isnan(padded_beacon_rewards).sum(axis=1)

line_and_bar_plot(np.nanmean(padded_utilities, axis=1)[:, queries], nan_count[:, queries], labels, title="Mean of Beacon Utility Level Among 30 Attacks", ylabel_line="Utility Level", ylabel_bar="Num of Predicted")

In [ ]:
import pickle

# Load the data from the pickle file
with open('/data6/sobhan/Beacons/results/SBA01.pkl', 'rb') as f:
    data_dict = pickle.load(f)

# Access each array
beacon_rewards = data_dict['beacon_rewards']
attacker_rewards = data_dict['attacker_rewards']
privacies = data_dict['privacies']
utilities = data_dict['utilities']
utilities2 = data_dict['utilities2']

padded_beacon_rewards = pad_to_max_length(beacon_rewards)[methods]
padded_attacker_rewards = pad_to_max_length(attacker_rewards)[methods]
padded_privacies = pad_to_max_length(privacies)[methods]
padded_utilities = pad_to_max_length(utilities2)[methods]
# labels = [ "SBA - OBD", "GBA - OBD", "SBA - GBD", "GBA - GBD",] 
nan_count = np.isnan(padded_beacon_rewards).sum(axis=1)

line_and_bar_plot(np.nanmean(padded_utilities, axis=1)[:, queries], nan_count[:, queries], labels, title="Mean of Beacon Utility Level Among 30 Attacks", ylabel_line="Utility Level", ylabel_bar="Num of Predicted")

In [ ]:
import pickle

# Load the data from the pickle file
with open('/data6/sobhan/Beacons/results/SBA.pkl', 'rb') as f:
    data_dict = pickle.load(f)

# Access each array
beacon_rewards = data_dict['beacon_rewards']
attacker_rewards = data_dict['attacker_rewards']
privacies = data_dict['privacies']
utilities = data_dict['utilities']
utilities2 = data_dict['utilities2']

padded_beacon_rewards = pad_to_max_length(beacon_rewards)[methods]
padded_attacker_rewards = pad_to_max_length(attacker_rewards)[methods]
padded_privacies = pad_to_max_length(privacies)[methods]
padded_utilities = pad_to_max_length(utilities2)[methods]
# labels = [ "SBA - OBD", "GBA - OBD", "SBA - GBD", "GBA - GBD",] 
nan_count = np.isnan(padded_beacon_rewards).sum(axis=1)

line_and_bar_plot(np.nanmean(padded_utilities, axis=1)[:, queries], nan_count[:, queries], labels, title="Mean of Beacon Utility Level Among 30 Attacks", ylabel_line="Utility Level", ylabel_bar="Num of Predicted")

In [ ]:
import pickle

# Load the data from the pickle file
with open('/data6/sobhan/Beacons/results/GBA.pkl', 'rb') as f:
    data_dict = pickle.load(f)

# Access each array
beacon_rewards = data_dict['beacon_rewards']
attacker_rewards = data_dict['attacker_rewards']
privacies = data_dict['privacies']
utilities = data_dict['utilities']
utilities2 = data_dict['utilities2']

padded_beacon_rewards = pad_to_max_length(beacon_rewards)[methods]
padded_attacker_rewards = pad_to_max_length(attacker_rewards)[methods]
padded_privacies = pad_to_max_length(privacies)[methods]
padded_utilities = pad_to_max_length(utilities2)[methods]
# labels = [ "SBA - OBD", "GBA - OBD", "SBA - GBD", "GBA - GBD",] 
nan_count = np.isnan(padded_beacon_rewards).sum(axis=1)

line_and_bar_plot(np.nanmean(padded_utilities, axis=1)[:, queries], nan_count[:, queries], labels, title="Mean of Beacon Utility Level Among 30 Attacks", ylabel_line="Utility Level", ylabel_bar="Num of Predicted")

In [ ]:
import pickle

# Load the data from the pickle file
with open('/data6/sobhan/Beacons/results/GBA01.pkl', 'rb') as f:
    data_dict = pickle.load(f)

# Access each array
beacon_rewards = data_dict['beacon_rewards']
attacker_rewards = data_dict['attacker_rewards']
privacies = data_dict['privacies']
utilities = data_dict['utilities']
utilities2 = data_dict['utilities2']

padded_beacon_rewards = pad_to_max_length(beacon_rewards)[methods]
padded_attacker_rewards = pad_to_max_length(attacker_rewards)[methods]
padded_privacies = pad_to_max_length(privacies)[methods]
padded_utilities = pad_to_max_length(utilities2)[methods]
# labels = [ "SBA - OBD", "GBA - OBD", "SBA - GBD", "GBA - GBD",] 
nan_count = np.isnan(padded_beacon_rewards).sum(axis=1)

line_and_bar_plot(np.nanmean(padded_utilities, axis=1)[:, queries], nan_count[:, queries], labels, title="Mean of Beacon Utility Level Among 30 Attacks", ylabel_line="Utility Level", ylabel_bar="Num of Predicted")

In [ ]:
import pickle

# Load the data from the pickle file
with open('/data6/sobhan/Beacons/results/random.pkl', 'rb') as f:
    data_dict = pickle.load(f)

# Access each array
beacon_rewards = data_dict['beacon_rewards']
attacker_rewards = data_dict['attacker_rewards']
privacies = data_dict['privacies']
utilities = data_dict['utilities']
utilities2 = data_dict['utilities2']

methods = [0, 1, 3, 4, 2, 6, 5]

padded_beacon_rewards = pad_to_max_length(beacon_rewards)[methods]
padded_attacker_rewards = pad_to_max_length(attacker_rewards)[methods]
padded_privacies = pad_to_max_length(privacies)[methods]
padded_utilities = pad_to_max_length(utilities2)[methods]
# labels = [ "SBA - OBD", "GBA - OBD", "SBA - GBD", "GBA - GBD",] 
nan_count = np.isnan(padded_beacon_rewards).sum(axis=1)

line_and_bar_plot(np.nanmean(padded_utilities, axis=1)[:, queries], nan_count[:,queries], labels, title="Mean of Beacon Utility Level Among 30 Attacks Against Optimal Attacker", ylabel_line="Utility Level", ylabel_bar="Num of Predicted")

In [ ]:

line_plot(np.nanmean(padded_utilities, axis=1)[:, queries], labels, title="Mean of Beacon Utility Level Among 30 Attacks Against Optimal Attacker", ylabel_line="Utility Level")

In [ ]:
import pickle

# Load the data from the pickle file
with open('/data6/sobhan/Beacons/results/simulate_01.pkl', 'rb') as f:
    data_dict = pickle.load(f)

# Access each array
beacon_rewards = data_dict['beacon_rewards']
attacker_rewards = data_dict['attacker_rewards']
privacies = data_dict['privacies']
utilities = data_dict['utilities']
utilities2 = data_dict['utilities2']

padded_beacon_rewards = pad_to_max_length(beacon_rewards)
padded_attacker_rewards = pad_to_max_length(attacker_rewards)
padded_privacies = pad_to_max_length(privacies)
padded_utilities = pad_to_max_length(utilities2)
labels = [ "SBA - OBD", "GBA - OBD", "SBA - GBD", "GBA - GBD",] 
nan_count = np.isnan(padded_beacon_rewards).sum(axis=1)
line_and_bar_plot(np.nanmean(padded_utilities, axis=1)[:, ::100], nan_count[:,::100], labels, title="Mean of Beacon Utility Level Among 30 Attacks", ylabel_line="Utility Level", ylabel_bar="Num of Predicted")

In [ ]:
padded_privacies = pad_to_max_length(privacies, value=0)

line_and_bar_plot(np.nanmean(padded_privacies, axis=1)[:, ::100], nan_count[:,::100], labels, title="Mean of Beacon Privacy Level Among 30 Attacks", ylabel_line="Privacy Level", ylabel_bar="Num of Predicted")


Scatter Plots

In [ ]:
padded_utilities = pad_to_max_length(utilities2, value=1)
# nan_count = (padded_utilities==0).sum(axis=1)
# nan_count = np.isnan(padded_utilities).sum(axis=1)
scatter_plot(np.nanmean(padded_utilities, axis=1)[:, ::100], nan_count[:,::100], labels, title="Mean of Beacon Honesty Level Among 20 Attacks", ylabel="Honesty Level")

In [ ]:
padded_utilities = pad_to_max_length(utilities, value=1)
# nan_count = (padded_utilities==0).sum(axis=1)
# nan_count = np.isnan(padded_utilities).sum(axis=1)
scatter_plot(np.nanmean(padded_utilities, axis=1)[:, ::100], nan_count[:,::100], labels, title="Mean of Beacon Honesty Level Among 20 Attacks", ylabel="Honesty Level")

In [ ]:
np.nanmean(padded_utilities, axis=1)[:, ::10]

In [ ]:
nan_count

In [ ]:
padded_privacies[0]

In [ ]:
padded_privacies = pad_to_max_length(privacies, value=0)
# nan_count = (padded_beacon_rewards==0).sum(axis=1)
labels = ["Truth", "Baseline 1%", "OBD", "Random 30%"] 
scatter_plot(np.nanmean(padded_privacies, axis=1)[:, ::100], nan_count[:,::100], labels, title="Mean of Privacies Among 20 Attacks", ylabel="Privacy (P-value)")

In [ ]:
plot_boxplot(padded_utilities[1:, :, ::100], labels, title="Distribution of Beacon Honesty Level Against the Optimal Attacker", ylabel="Honesty Level")

In [ ]:
plot_boxplot(padded_privacies[:, :, ::100], labels, title="Distribution of Victim Privacy Against the Optimal Attacker", ylabel="Privacy Level (P-value)")

In [ ]:
padded_attacker_rewards = np.clip(pad_to_max_length(attacker_rewards, value=0), None, 0)

# padded_attacker_rewards +=1
nan_count = (padded_beacon_rewards==0).sum(axis=1)
scatter_plot(np.nanmean(padded_attacker_rewards, axis=1)[:, ::10], nan_count[:,::10], labels, title="Mean of Attacker Rewards Among 20 Attacks", ylabel="Reward")

Line Plots

In [ ]:
padded_beacon_rewards = pad_to_max_length(beacon_rewards, value=0)
line_plot(np.mean(padded_beacon_rewards, axis=1)[:, ::9], labels, title="Mean of Beacon Rewards Among 20 Attacks", ylabel="Reward")

In [ ]:
padded_beacon_rewards = pad_to_max_length(beacon_rewards, value=np.nan)
line_plot(np.nanmean(padded_beacon_rewards, axis=1)[:, ::9], labels, title="Mean of Beacon Rewards Among 20 Attacks", ylabel="Reward")

In [ ]:
padded_attacker_rewards = np.clip(pad_to_max_length(attacker_rewards, value=np.nan), None, 0)
padded_attacker_rewards[padded_attacker_rewards < -2] += 5
scatter_plot(np.nanmean(padded_attacker_rewards, axis=1)[:4, ::9], nan_count[:,::9], labels, title="Mean of Attacker Rewards Among 20 Attacks", ylabel="Reward")

In [ ]:
line_plot(np.mean(padded_utilities, axis=1)[:, ::9], labels, title="Mean of Beacon Truth Level Among 20 Attacks", ylabel="Truth Level")

In [ ]:
padded_utilities = pad_to_max_length(utilities, value=np.nan)
scatter_plot(np.nanmean(padded_utilities, axis=1)[:3, ::9], nan_count[:,::9], labels, title="Mean of Beacon Truth Level Among 20 Attacks", ylabel="Truth Level")

In [ ]:
padded_privacies = pad_to_max_length(privacies, value=np.nan)
scatter_plot(np.nanmean(padded_privacies, axis=1)[:3, ::9], nan_count[:,::9], labels, title="Mean of Beacon Truth Level Among 20 Attacks", ylabel="Truth Level")

In [ ]:
plot_boxplot(padded_utilities[:, :, ::100], labels, title="Distribution of Beacon Rewards Against the Agent Attacker", ylabel="Reward")
plot_violinplot(padded_utilities[:, :, ::100], labels, title="Distribution of Beacon Rewards Against the Agent Attacker", ylabel="Reward")

In [ ]:
plot_boxplot(np.clip(padded_beacon_rewards[:, :, ::9], 0, None), labels, title="Distribution of Beacon Rewards Against the Agent Attacker", ylabel="Reward")
plot_violinplot(np.clip(padded_beacon_rewards[:, :, ::9], 0, None), labels, title="Distribution of Beacon Rewards Against the Agent Attacker", ylabel="Reward")

In [ ]:
# plot_violinplot(np.clip(padded_beacon_rewards[:, :, ::9], 0, None), labels, title="Distribution of Beacon Rewards Against the Agent Attacker", ylabel="Reward")
plot_violinplot(np.clip(padded_beacon_rewards[:, :, ::9], 0, None), labels, title="Distribution of Beacon Rewards Against the Agent Attacker", ylabel="Reward", scatter_data = nan_count[:,::9])

In [ ]:
plot_boxplot(np.clip(padded_attacker_rewards[:, :, ::9], None, 0), labels, title="Distribution of Attacker Rewards Against the Agent Attacker", ylabel="Reward")
plot_violinplot(np.clip(padded_attacker_rewards[:, :, ::9], None, 0), labels, title="Distribution of Attacker Rewards Against the Agent Attacker", ylabel="Reward")

In [ ]:
padded_attacker_rewards = pad_to_max_length(attacker_rewards, value=np.nan)

padded_attacker_rewards[padded_attacker_rewards < -2] += 5
padded_attacker_rewards[1, :, :], padded_attacker_rewards[2, :, :] = padded_attacker_rewards[2, :, :], padded_attacker_rewards[1, :, :].copy()
labels = ["Optimal Attacker-Beacon Agent B", "Agent Attacker-Beacon Agent B",]  # Define your labels
plot_boxplot(np.clip(padded_attacker_rewards[1::2, :, ::9], None, 0), labels, title="Distribution of Attacker Rewards Against the Agent Attacker", ylabel="Reward")
plot_violinplot(np.clip(padded_attacker_rewards[1::2, :, ::9], None, 0), labels, title="Distribution of Attacker Rewards Against the Agent Attacker", ylabel="Reward")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


params = {'legend.fontsize': 48,
        'figure.figsize': (54, 32),
        'axes.labelsize': 60,
        'axes.titlesize':60,
        'xtick.labelsize':60,
        'ytick.labelsize':60,
        'lines.linewidth': 10}

plt.rcParams.update(params)

# Create sample data (two arrays)
data1 = np.random.normal(60, 10, size=100)
data2 = np.random.normal(65, 15, size=100)
data3 = np.random.normal(55, 8, size=100)

# Combine the data into one dataset
data = [data1, data2, data3]
labels = ['Generated', 'Natural']




def plot_violin_compare(natural, generated, labels, filename):
    # Create the violin plot
    plt.figure()
    data = [natural, generated]
    sns.violinplot(data=data, inner='box', palette=['#4F6367', '#A4B494', '#D4A5A5'])

    # Customizing the plot to resemble the example
    plt.xticks([0, 1], labels)
    plt.yticks()
    # plt.ylim(40, 90)
    plt.ylabel('', )
    plt.title('A',loc='left')

    # Show plot
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()
    # plt.show()

plot_violin_compare(data1, data2, labels)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def violin_plot(data, edge_color, fill_color, position, ax):
    parts = ax.violinplot(data, positions=[position], widths=0.5, showmedians=True)
    
    for pc in parts['bodies']:
        pc.set_facecolor(fill_color)
        pc.set_edgecolor(edge_color)
        pc.set_linewidth(1)
        pc.set_alpha(1)
    
    if 'cmedians' in parts:
        parts['cmedians'].set_color(edge_color)
        parts['cmedians'].set_linewidth(1)
    
    for partname in ('cbars', 'cmins', 'cmaxes'):
        vp = parts[partname]
        vp.set_edgecolor(edge_color)
        vp.set_linewidth(1)


def plot_violinplot(data, scatter_data, labels, title, ylabel):

    def scatter_plot(ax, x_positions, y_data, size_data, color_palette, labels):
        # Define marker size groups
        size_ranges = [(1, 5), (5, 10), (10, 15), (15, 20)]
        marker_sizes = [200, 400, 600, 800]  # Corresponding marker sizes for the ranges
        size_labels = ["1-5", "5-10", "10-15", "15-20"]  # Labels for legend

        classes, stages = y_data.shape

        # Loop through each class
        for class_idx in range(classes):
            y_class_data = y_data[class_idx, :]  # y-axis data for the class
            size_class_data = size_data[class_idx, :]  # Size data for the class
            edge_color = 'black'
            marker_color = color_palette[class_idx]
            marker_style = 'o'

            # Assign marker sizes based on the size ranges
            marker_sizes_class = np.zeros_like(size_class_data)
            for i, size_range in enumerate(size_ranges):
                mask = (size_class_data >= size_range[0]) & (size_class_data < size_range[1])
                marker_sizes_class[mask] = marker_sizes[i]

            # Scatter plot with custom marker sizes
            ax.scatter(x_positions[class_idx], y_class_data, s=marker_sizes_class,
                    color=marker_color, edgecolor=edge_color, marker=marker_style, linewidth=1.5)
            # ax.plot(x_positions[class_idx], y_class_data, color=marker_color, linewidth=5)

        # Legend for the marker sizes
        size_handles = [plt.Line2D([], [], color='w', marker='o', markersize=np.sqrt(size/2),
                                markerfacecolor='gray', label=label, markeredgecolor='black')
                        for size, label in zip(marker_sizes, size_labels)]
        legend1 = ax.legend(size_handles, size_labels, title="Num. Identified", loc="lower left", title_fontsize=12)
        ax.add_artist(legend1)
    """
    Generates a violin plot with overlayed scatter points based on a secondary array for marker sizes.
    
    Parameters:
    data (numpy array): A 3D numpy array with shape (classes, samples, stages).
    scatter_data (numpy array): A 2D numpy array with shape (classes, stages) for marker sizes.
    """
    color_palette = ["#65879F", "#8B8C89", "#425062", "#8F5C5C", "#CFACAC"]
    classes, samples, stages = data.shape
    
    fig, ax = plt.subplots()

    # Adjust x-positions for each stage and class
    x_positions = []
    for stage in range(stages):
        x_positions.append(np.arange(classes) + stage * (classes + 1))

    x_positions = np.array(x_positions).T  # Transpose to match class and stage positions

    for class_idx in range(classes):
        class_data = data[class_idx]  # shape (samples, stages)
        
        for stage in range(stages):
            stage_data = class_data[:, stage]
            position = x_positions[class_idx, stage]
            edge_color = 'black'
            fill_color = color_palette[class_idx]
            
            violin_plot(stage_data, edge_color, fill_color, position, ax)
    
    # Overlay scatter plot for marker sizes using the median of each stage
    scatter_plot(ax, x_positions, np.median(data, axis=1), scatter_data, color_palette, labels)

    ax.set_xlabel('Query')
    ax.set_ylabel(ylabel)
    ax.set_title(title)

    # Setting the class labels legend
    class_handles = [mpatches.Patch(color=color_palette[i], label=labels[i], edgecolor='black') for i in range(classes)]
    ax.legend(class_handles, labels, loc="center right")
    
    plt.show()

# Sample Data
np.random.seed(42)
data = np.random.rand(5, 19, 6) * 50  # 5 classes, 19 samples, 6 stages
scatter_data = np.random.randint(0, 20, (5, 6))  # 5 classes, 6 stages

# Labels
labels = [f'Class {i+1}' for i in range(5)]

# Run the plot
plot_violinplot(data, scatter_data, labels, "Violin Plot with Scatter", "Value")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate a sample 3D numpy array of size (5, 19, 6) for the violin plot
np.random.seed(42)
data = np.random.rand(5, 19, 6) * 50  # 5 classes, 19 samples, 6 stages

# Generate a sample 2D numpy array of size (5, 6) for scatter marker sizes
scatter_data = np.random.randint(0, 20, (5, 6))  # 5 classes, 6 stages

# Define labels for the classes
labels = [f'Class {i+1}' for i in range(5)]

# Run the plot function
plot_violinplot(data, scatter_data, labels, "Violin Plot with Scatter", "Value")


In [3]:

params = {'legend.fontsize': 48,
        'figure.figsize': (54, 32),
        'axes.labelsize': 60,
        'axes.titlesize':60,
        'xtick.labelsize':60,
        'ytick.labelsize':60,
        'lines.linewidth': 10}

plt.rcParams.update(params)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def line_and_bar_plot(line_data, bar_data, labels, title, ylabel_line, ylabel_bar):
    fig, ax1 = plt.subplots()
    color_palette = ["#65879F", "#8B8C89", "#425062", "#8F5C5C", "#CFACAC"]
    x_ticks = ['1', '10', '20', '30', '40', '50']
    classes, stages = line_data.shape
    
    x_positions = np.arange(stages)
    bar_width = 0.1  # Bar width for the bar plot
    
    # Plotting the line data
    for class_idx in range(classes):
        class_data = line_data[class_idx, :]  # shape (stages,)
        edge_color = 'black'
        line_color = color_palette[class_idx]
        
        # Plotting the line with markers
        ax1.plot(x_positions, class_data, marker='o', color=line_color, markeredgewidth=5, markeredgecolor=edge_color, linewidth=5)

    ax1.set_xlabel('Query')
    ax1.set_ylabel(ylabel_line)
    ax1.set_title(title)
    
    # Set x-axis ticks and labels
    ax1.set_xticks(x_positions)
    ax1.set_xticklabels(x_ticks)
    ax1.grid(True, linestyle='--', color='gray', alpha=0.5)
    
    # Adding legend for line plot
    handles = [mpatches.Patch(color=color_palette[i], label=labels[i], edgecolor='black') for i in range(classes)]
    ax1.legend(handles, labels, loc="lower right")
    
    # Create a second y-axis for the bar plot
    ax2 = ax1.twinx()
    for class_idx in range(classes):
        bar_vals = bar_data[class_idx, :]  # shape (stages,)
        bar_color = color_palette[class_idx]
        
        # Offset the bar positions to avoid overlap
        ax2.bar(x_positions + (class_idx - classes / 2) * bar_width, bar_vals, width=bar_width, color=bar_color, alpha=0.6, label=f'Bar {labels[class_idx]}')

    # Set ylabel for bar plot
    ax2.set_ylabel(ylabel_bar)
    
    # Adding legend for bar plot
    bar_handles = [mpatches.Patch(color=color_palette[i], label=f'Bar {labels[i]}') for i in range(classes)]
    ax2.legend(bar_handles, labels, loc="upper right")

    plt.show()

# Example Usage
line_data = np.random.random((5, 6)) * 2  # Example data for line plot (5 classes, 6 stages)
bar_data = np.random.random((5, 6)) * 3  # Example data for bar plot (5 classes, 6 stages)
labels = ["Strategic", "Truth", "Baseline 1%", "OBD", "Random 30%"] 
title = "Combined Line and Bar Plot"
ylabel_line = "Line Plot Y-Label"
ylabel_bar = "Bar Plot Y-Label"

line_and_bar_plot(line_data, bar_data, labels, title, ylabel_line, ylabel_bar)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def line_and_bar_plot(line_data, bar_data, labels, title, ylabel_line, ylabel_bar):
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [3, 1]})
    
    color_palette = ["#65879F", "#8B8C89", "#425062", "#8F5C5C", "#CFACAC"]
    x_ticks = ['100', '200', '300', '400', '500', '600', '700', '800', '900', '1000']
    classes, stages = line_data.shape
    
    x_positions = np.arange(stages)
    bar_width = 0.1  # Bar width for the bar plot
    
    # Plotting the line data
    for class_idx in range(classes):
        class_data = line_data[class_idx, :]  # shape (stages,)
        edge_color = 'black'
        line_color = color_palette[class_idx]
        
        # Plotting the line with markers
        ax1.plot(x_positions, class_data, marker='o', color=line_color, markeredgewidth=5, markeredgecolor=edge_color, linewidth=5)

    ax1.set_ylabel(ylabel_line)
    ax1.set_title(title)
    
    # Set x-axis ticks and labels
    ax1.set_xticks(x_positions)
    ax1.set_xticklabels(x_ticks)
    ax1.grid(True, linestyle='--', color='gray', alpha=0.5)
    
    # Adding legend for line plot
    handles = [mpatches.Patch(color=color_palette[i], label=labels[i], edgecolor='black') for i in range(classes)]
    ax1.legend(handles, labels, loc="upper left")
    
    # Plotting the bar data on the lower axis
    for class_idx in range(classes):
        bar_vals = bar_data[class_idx, :]  # shape (stages,)
        bar_color = color_palette[class_idx]
        
        # Offset the bar positions to avoid overlap
        ax2.bar(x_positions + (class_idx - classes / 2) * bar_width, bar_vals, width=bar_width, color=bar_color, alpha=0.6)

    # Set ylabel for bar plot
    ax2.set_ylabel(ylabel_bar)
    ax2.set_xlabel('Query')
    
    # Set x-axis ticks and labels for the bar plot (shared with line plot)
    ax2.set_xticks(x_positions)
    ax2.set_xticklabels(x_ticks)
    
    plt.tight_layout()
    plt.show()

# Example Usage
line_data = np.random.random((5, 6)) * 2  # Example data for line plot (5 classes, 6 stages)
bar_data = np.random.random((5, 6)) * 3  # Example data for bar plot (5 classes, 6 stages)
labels = ["Strategic", "Truth", "Baseline 1%", "OBD", "Random 30%"] 
title = "Combined Line and Bar Plot"
ylabel_line = "Line Plot Y-Label"
ylabel_bar = "Bar Plot Y-Label"

line_and_bar_plot(np.nanmean(padded_utilities, axis=1)[:, ::100], nan_count[:,::100], labels, title="Mean of Beacon Honesty Level Among 20 Attacks", ylabel_line="Honesty Level", ylabel_bar="Num of Predicted")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def line_and_bar_plot(line_data, bar_data, labels, title, ylabel_line, ylabel_bar):
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [3, 1]})
    
    color_palette = ["#65879F", "#8B8C89", "#425062", "#8F5C5C", "#CFACAC", "#CFACAC", "#CFACAC"]
    x_ticks = ['100', '200', '300', '400', '500', '600', '700', '800', '900', '1000']
    classes, stages = line_data.shape
    
    x_positions = np.arange(stages)
    bar_width = 0.15  # Adjusted Bar width for better clarity
    
    # Plotting the line data
    for class_idx in range(classes):
        class_data = line_data[class_idx, :]  # shape (stages,)
        edge_color = 'black'
        line_color = color_palette[class_idx]
        
        # Plotting the line with markers
        ax1.plot(x_positions, class_data, marker='o', color=line_color, markeredgewidth=5, markeredgecolor=edge_color, linewidth=5)

    ax1.set_ylabel(ylabel_line)
    ax1.set_title(title)
    
    # Set x-axis ticks and labels
    ax1.set_xticks(x_positions)
    ax1.set_xticklabels(x_ticks)
    ax1.grid(True, linestyle='--', color='gray', alpha=0.5)
    
    # Adding legend for line plot
    handles = [mpatches.Patch(color=color_palette[i], label=labels[i], edgecolor='black') for i in range(classes)]
    ax1.legend(handles, labels, loc="upper left")
    
    # Plotting the bar data on the lower axis
    for class_idx in range(classes):
        bar_vals = bar_data[class_idx, :]  # shape (stages,)
        bar_color = color_palette[class_idx]
        
        # Offset the bar positions to avoid overlap
        bars = ax2.bar(x_positions + (class_idx - classes / 2) * bar_width, bar_vals, width=bar_width, color=bar_color, alpha=0.6)
        
        # Annotate bars with their values
        for bar in bars:
            height = bar.get_height()
            ax2.annotate(f'{height:.1f}',  # format the label
                         xy=(bar.get_x() + bar.get_width() / 2, height),  # set the label position
                         xytext=(0, 3),  # 3 points vertical offset
                         textcoords="offset points",
                         ha='center', va='bottom',
                         fontsize=8)

    # Set ylabel for bar plot
    ax2.set_ylabel(ylabel_bar)
    ax2.set_xlabel('Query')
    
    # Set x-axis ticks and labels for the bar plot (shared with line plot)
    ax2.set_xticks(x_positions)
    ax2.set_xticklabels(x_ticks)
    
    plt.tight_layout()
    plt.show()

# Example Usage
line_data = np.random.random((5, 10)) * 2  # Example data for line plot (5 classes, 6 stages)
bar_data = np.random.random((5, 10)) * 3  # Example data for bar plot (5 classes, 6 stages)
labels = ["Strategic", "Truth", "Baseline 1%", "OBD", "Random 30%"] 
title = "Combined Line and Bar Plot"
ylabel_line = "Line Plot Y-Label"
ylabel_bar = "Bar Plot Y-Label"
# Example Usage with your data
# Assuming the line_data and bar_data are already defined and prepared
line_and_bar_plot(line_data,bar_data, labels, title, ylabel_line,ylabel_bar )